In [1]:
pip install pyarrow


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers pyarrow


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install fpdf


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install wordcloud


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pillow


In [3]:
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\alamm\AppData\Roaming\nltk_data...


True

In [ ]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import time
import os
from datetime import datetime
from fpdf import FPDF
from threading import Thread
from pathlib import Path
import pygetwindow as gw
import re

# Global variables
activity_log = []  # List to store activities
is_monitoring = False
eye_window = None

# Get active window
def get_active_window():
    try:
        windows = gw.getWindowsWithTitle("")
        for window in windows:
            if window.isActive:
                return window.title
    except Exception as e:
        return "Unknown"
    return "None"

# Start monitoring
def start_monitoring():
    global is_monitoring
    if not is_monitoring:
        is_monitoring = True
        Thread(target=log_activity).start()
        messagebox.showinfo("Monitoring", "Monitoring started!")
    else:
        messagebox.showinfo("Monitoring", "Monitoring is already running.")

# Stop monitoring
def stop_monitoring():
    global is_monitoring
    if is_monitoring:
        is_monitoring = False
        messagebox.showinfo("Monitoring", "Stopped monitoring activities.")
    else:
        messagebox.showinfo("Monitoring", "Monitoring is not active.")

# Log activity
def log_activity():
    while is_monitoring:
        active_window = get_active_window()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        activity_log.append({"time": timestamp, "window": active_window})
        time.sleep(5)  # Monitor every 5 seconds

# Sanitize text for PDF
def sanitize_text(text):
    """
    Removes characters that are not supported by the 'latin-1' codec.
    """
    return re.sub(r'[^\x00-\xFF]', '', text)

# Generate PDF report
def generate_report():
    if not activity_log:
        messagebox.showinfo("Report", "No activity to report.")
        return

    try:
        # Create PDF
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)

        # Add title
        pdf.set_font("Arial", style="B", size=16)
        title = sanitize_text("Activity Monitoring Report")
        pdf.cell(200, 10, txt=title, ln=True, align='C')

        # Add table headers
        pdf.set_font("Arial", size=12)
        pdf.cell(80, 10, txt="Time", border=1)
        pdf.cell(110, 10, txt="Active Window", border=1)
        pdf.ln()

        # Add activity data
        for entry in activity_log:
            pdf.cell(80, 10, txt=sanitize_text(entry["time"]), border=1)
            pdf.cell(110, 10, txt=sanitize_text(entry["window"][:30]), border=1)
            pdf.ln()

        # Save the PDF to the Downloads folder with a unique name
        downloads_path = str(Path.home() / "Downloads")
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = os.path.join(downloads_path, f"Activity_Report_{timestamp}.pdf")
        pdf.output(filename)

        # Confirm file creation
        if os.path.exists(filename):
            messagebox.showinfo("Report", f"Report generated successfully: {filename}")
        else:
            messagebox.showerror("Report Error", "Failed to create the PDF file. Please check permissions.")

    except Exception as e:
        # Log and alert on errors
        print(f"Error generating report: {e}")
        messagebox.showerror("Report Error", f"Failed to generate report: {e}")

# Update GIF frames dynamically
def update_gif(label, gif_frames, frame_index):
    frame = gif_frames[frame_index]
    label.config(image=frame)
    label.image = frame  # Keep a reference to avoid garbage collection
    next_frame = (frame_index + 1) % len(gif_frames)
    label.after(100, update_gif, label, gif_frames, next_frame)  # 100ms delay for next frame

# Create floating window with GIF and buttons
def create_floating_eye():
    global eye_window
    eye_window = tk.Toplevel()
    eye_window.title("Monitoring Eye")
    eye_window.geometry("350x300")
    eye_window.attributes("-topmost", True)
    eye_window.overrideredirect(True)

    # Place the window at the top-right corner
    x_position = eye_window.winfo_screenwidth() - 360
    y_position = 10
    eye_window.geometry(f"350x300+{x_position}+{y_position}")

    # Load the animated GIF
    gif_path = "C:/Users/alamm/Downloads/eye.gif"  # Update with your GIF path
    gif_image = Image.open(gif_path)
    
    # Extract frames from the GIF
    gif_frames = []
    try:
        while True:
            gif_frame = gif_image.copy()
            gif_frame = gif_frame.resize((350, 200))  # Resize to fit the window
            gif_frames.append(ImageTk.PhotoImage(gif_frame))
            gif_image.seek(len(gif_frames))  # Move to the next frame
    except EOFError:
        pass  # End of frames

    # Create a Label widget to hold the GIF
    gif_label = tk.Label(eye_window)
    gif_label.pack()

    # Start updating the GIF
    if gif_frames:
        update_gif(gif_label, gif_frames, 0)

    # Add buttons on top of the GIF
    button_frame = tk.Frame(eye_window, bg="black", relief=tk.RAISED)
    button_frame.place(relx=0.5, rely=0.85, anchor=tk.CENTER)

    tk.Button(button_frame, text="Start Monitoring", command=start_monitoring).pack(side=tk.LEFT, padx=10)
    tk.Button(button_frame, text="Stop Monitoring", command=stop_monitoring).pack(side=tk.LEFT, padx=10)
    tk.Button(button_frame, text="Generate Report", command=generate_report).pack(side=tk.LEFT, padx=10)

# Main application entry
def main_app():
    global main_window
    main_window = tk.Tk()
    main_window.withdraw()  # Hide the main window

    # Start the floating window with the eye GIF
    create_floating_eye()
    main_window.mainloop()

# Run the app
if __name__ == "__main__":
    main_app()
